# 📊 Stock Price Prediction - Part 1: Data & AI Model

## FÁZE 2-3: Stažení Fundamentálních Dat a Trénování AI Modelu

**Autor:** Bc. Jan Dub  
**Datum:** Říjen 2025  
**Google Colab Ready** ✅

---

### 🎯 Cíl tohoto notebooku:

1. **FÁZE 2:** Stáhnout fundamentální metriky (P/E, ROE, atd.) pro období 2024-2025
2. **FÁZE 3:** Natrénovat Random Forest model, který predikuje fundamenty z OHLCV dat

### 📝 Co budeme dělat:

- Nahrát OHLCV data z Google Drive
- Stáhnout fundamentální data pomocí yfinance
- Spojit OHLCV + fundamenty
- Natrénovat Multi-output Random Forest
- Evaluovat model (MAE, RMSE, R²)
- Analyzovat feature importance
- Uložit model pro další použití

## 📦 1. Instalace a Import Knihoven

In [ ]:
# Instalace potřebných knihoven (v Colabu)
!pip install -q yfinance scikit-learn joblib matplotlib seaborn

In [ ]:
# Import základních knihoven
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import time
from datetime import datetime

# Machine Learning
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from joblib import dump, load

# yfinance pro fundamentální data
import yfinance as yf

# Nastavení
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Knihovny načteny")

## ⚙️ 2. Konfigurace

In [ ]:
# === KONFIGURACE ===

# Časové období pro fundamentální data
START_DATE = "2024-01-01"
END_DATE = "2025-10-31"

# Features z OHLCV dat
OHLCV_FEATURES = [
    'open', 'high', 'low', 'close', 'volume',
    'volatility', 'returns',
    'rsi_14', 'macd', 'macd_signal', 'macd_hist',
    'sma_3', 'sma_6', 'sma_12',
    'ema_3', 'ema_6', 'ema_12',
    'volume_change'
]

# Target fundamentální metriky
FUNDAMENTAL_TARGETS = [
    'PE', 'PB', 'PS', 'EV_EBITDA',  # Valuační
    'ROE', 'ROA', 'Profit_Margin', 'Operating_Margin', 'Gross_Margin',  # Profitabilita
    'Debt_to_Equity', 'Current_Ratio', 'Quick_Ratio',  # Finanční zdraví
    'Revenue_Growth_YoY', 'Earnings_Growth_YoY'  # Růst
]

# Hyperparametry Random Forest
RF_PARAMS = {
    'n_estimators': 100,
    'max_depth': 20,
    'min_samples_split': 5,
    'min_samples_leaf': 2,
    'random_state': 42,
    'n_jobs': -1
}

print(f"✅ Konfigurace nastavena")
print(f"   • Období: {START_DATE} → {END_DATE}")
print(f"   • OHLCV features: {len(OHLCV_FEATURES)}")
print(f"   • Fundamental targets: {len(FUNDAMENTAL_TARGETS)}")

## 📂 3. Načtení OHLCV Dat

### Google Drive Upload:
1. Nahrajte soubor `all_sectors_full_10y.csv` do Google Drive
2. Připojte Google Drive v Colabu
3. Spusťte následující buňku

In [ ]:
# Připojení Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Cesta k datům (upravte podle vaší struktury)
OHLCV_PATH = '/content/drive/MyDrive/StockPrediction/all_sectors_full_10y.csv'

print("✅ Google Drive připojen")

In [ ]:
# Načtení OHLCV dat
print("📂 Načítám OHLCV data...")

ohlcv_df = pd.read_csv(OHLCV_PATH)
ohlcv_df['date'] = pd.to_datetime(ohlcv_df['date'])

print(f"✅ Načteno {len(ohlcv_df):,} záznamů")
print(f"   • Období: {ohlcv_df['date'].min()} → {ohlcv_df['date'].max()}")
print(f"   • Tickery: {ohlcv_df['ticker'].nunique()}")
print(f"   • Sektory: {', '.join(ohlcv_df['sector'].unique())}")

# Náhled dat
display(ohlcv_df.head())

## 📥 4. FÁZE 2: Stažení Fundamentálních Dat

Stáhneme fundamentální metriky pomocí yfinance pro období 2024-2025.

In [ ]:
# Pomocné funkce pro stahování fundamentů

def safe_get_info(ticker, key, default=None):
    """Bezpečně získá hodnotu z ticker.info"""
    try:
        info = ticker.info
        return info.get(key, default)
    except:
        return default

def get_col(df, candidates):
    """Vrátí první existující sloupec z candidates"""
    for col in candidates:
        if col in df.columns:
            return df[col]
    return pd.Series(np.nan, index=df.index)

def calculate_quarterly_fundamentals(ticker_str):
    """Stáhne a vypočítá fundamentální metriky z quarterly dat"""
    try:
        ticker = yf.Ticker(ticker_str)
        
        # Základní info
        market_cap = safe_get_info(ticker, 'marketCap')
        shares_outstanding = safe_get_info(ticker, 'sharesOutstanding')
        
        # Quarterly statements
        financials = ticker.quarterly_financials.T
        balance_sheet = ticker.quarterly_balance_sheet.T
        
        if financials.empty or balance_sheet.empty:
            return pd.DataFrame()
        
        # Align indexy
        financials.index = pd.to_datetime(financials.index)
        balance_sheet.index = pd.to_datetime(balance_sheet.index)
        
        # Merge
        df = financials.join(balance_sheet, how='outer', rsuffix='_bs')
        df = df[(df.index >= START_DATE) & (df.index <= END_DATE)]
        
        if df.empty:
            return pd.DataFrame()
        
        # Extrakce dat
        total_revenue = get_col(df, ['Total Revenue', 'TotalRevenue'])
        net_income = get_col(df, ['Net Income', 'NetIncome'])
        ebitda = get_col(df, ['EBITDA', 'Ebitda'])
        operating_income = get_col(df, ['Operating Income', 'OperatingIncome'])
        gross_profit = get_col(df, ['Gross Profit', 'GrossProfit'])
        
        total_equity = get_col(df, ['Total Stockholder Equity', 'Stockholders Equity'])
        total_assets = get_col(df, ['Total Assets', 'TotalAssets'])
        total_debt = get_col(df, ['Total Debt', 'Long Term Debt'])
        current_assets = get_col(df, ['Current Assets', 'CurrentAssets'])
        current_liabilities = get_col(df, ['Current Liabilities', 'CurrentLiabilities'])
        cash = get_col(df, ['Cash And Cash Equivalents', 'Cash'])
        
        # TTM
        revenue_ttm = total_revenue.rolling(4, min_periods=1).sum()
        net_income_ttm = net_income.rolling(4, min_periods=1).sum()
        ebitda_ttm = ebitda.rolling(4, min_periods=1).sum()
        operating_income_ttm = operating_income.rolling(4, min_periods=1).sum()
        gross_profit_ttm = gross_profit.rolling(4, min_periods=1).sum()
        
        # Vypočítané metriky
        result = pd.DataFrame(index=df.index)
        result['ticker'] = ticker_str
        
        # Valuace
        if market_cap and shares_outstanding:
            approx_price = market_cap / shares_outstanding
            eps_ttm = net_income_ttm / shares_outstanding
            result['PE'] = approx_price / eps_ttm.replace(0, np.nan)
            result['PS'] = market_cap / revenue_ttm.replace(0, np.nan)
        else:
            result['PE'] = np.nan
            result['PS'] = np.nan
        
        result['PB'] = (market_cap if market_cap else np.nan) / total_equity.replace(0, np.nan)
        
        # EV/EBITDA
        if market_cap:
            ev = market_cap + total_debt.fillna(0) - cash.fillna(0)
            result['EV_EBITDA'] = ev / ebitda_ttm.replace(0, np.nan)
        else:
            result['EV_EBITDA'] = np.nan
        
        # Profitabilita
        result['ROE'] = net_income_ttm / total_equity.replace(0, np.nan)
        result['ROA'] = net_income_ttm / total_assets.replace(0, np.nan)
        result['Profit_Margin'] = net_income_ttm / revenue_ttm.replace(0, np.nan)
        result['Operating_Margin'] = operating_income_ttm / revenue_ttm.replace(0, np.nan)
        result['Gross_Margin'] = gross_profit_ttm / revenue_ttm.replace(0, np.nan)
        
        # Finanční zdraví
        result['Debt_to_Equity'] = total_debt / total_equity.replace(0, np.nan)
        result['Current_Ratio'] = current_assets / current_liabilities.replace(0, np.nan)
        result['Quick_Ratio'] = (current_assets - get_col(df, ['Inventory'])) / current_liabilities.replace(0, np.nan)
        
        # Růst
        result['Revenue_Growth_YoY'] = revenue_ttm.pct_change(periods=4)
        result['Earnings_Growth_YoY'] = net_income_ttm.pct_change(periods=4)
        
        return result
        
    except Exception as e:
        print(f"  ❌ {ticker_str}: {e}")
        return pd.DataFrame()

print("✅ Funkce pro stahování fundamentů připraveny")

In [ ]:
# Stažení fundamentálních dat pro všechny tickery

print("📥 Stahuji fundamentální data...")
print(f"   Období: {START_DATE} → {END_DATE}")

tickers = ohlcv_df['ticker'].unique()
print(f"   Tickery: {len(tickers)}")

all_fundamentals = []
errors = 0

for i, ticker in enumerate(tickers, 1):
    if i % 10 == 0:
        print(f"   [{i}/{len(tickers)}] {ticker}...")
    
    df = calculate_quarterly_fundamentals(ticker)
    
    if not df.empty:
        # Přidat sektor
        sector = ohlcv_df[ohlcv_df['ticker'] == ticker]['sector'].iloc[0]
        df['sector'] = sector
        all_fundamentals.append(df)
    else:
        errors += 1
    
    time.sleep(0.3)  # Rate limiting

# Spojení všech dat
if all_fundamentals:
    fundamentals_df = pd.concat(all_fundamentals, ignore_index=False)
    fundamentals_df = fundamentals_df.reset_index().rename(columns={'index': 'date'})
    
    print(f"\n✅ Staženo {len(fundamentals_df)} záznamů")
    print(f"   • Úspěšné tickery: {len(tickers) - errors}")
    print(f"   • Chyby: {errors}")
    
    display(fundamentals_df.head())
else:
    print("❌ Žádná data ke stažení")

## 🔗 5. Spojení OHLCV a Fundamentálních Dat

In [ ]:
# Spojení OHLCV a fundamentálních dat

print("🔗 Spojuji OHLCV a fundamentální data...")

# Pro každý ticker zvlášť s forward-fill
merged_parts = []

for ticker in fundamentals_df['ticker'].unique():
    # OHLCV pro ticker
    ohlcv_ticker = ohlcv_df[ohlcv_df['ticker'] == ticker].copy()
    ohlcv_ticker = ohlcv_ticker.sort_values('date').set_index('date')
    
    # Fundamenty pro ticker
    fund_ticker = fundamentals_df[fundamentals_df['ticker'] == ticker].copy()
    fund_ticker = fund_ticker.sort_values('date').set_index('date')
    
    # Merge s forward-fill
    merged = ohlcv_ticker.join(fund_ticker[FUNDAMENTAL_TARGETS], how='left')
    merged[FUNDAMENTAL_TARGETS] = merged[FUNDAMENTAL_TARGETS].fillna(method='ffill')
    
    merged = merged.reset_index()
    merged_parts.append(merged)

merged_df = pd.concat(merged_parts, ignore_index=True)

# Filtrovat pouze období kde máme fundamenty (2024-2025)
merged_df = merged_df[merged_df['date'] >= '2024-01-01'].copy()

# Odstranit chybějící hodnoty
merged_df = merged_df.dropna(subset=OHLCV_FEATURES + FUNDAMENTAL_TARGETS)

print(f"✅ Spojeno: {len(merged_df):,} záznamů")
print(f"   • Tickery: {merged_df['ticker'].nunique()}")
print(f"   • Období: {merged_df['date'].min()} → {merged_df['date'].max()}")

display(merged_df.head())

## 🤖 6. FÁZE 3: Trénování AI Modelu

Natrénujeme Multi-output Random Forest model.

In [ ]:
# Příprava trénovacích dat

print("🔧 Příprava trénovacích dat...")

X = merged_df[OHLCV_FEATURES].copy()
y = merged_df[FUNDAMENTAL_TARGETS].copy()

# Odstranit nekonečné hodnoty
X = X.replace([np.inf, -np.inf], np.nan)
y = y.replace([np.inf, -np.inf], np.nan)

# Dropnout NaN
valid_mask = ~(X.isna().any(axis=1) | y.isna().any(axis=1))
X = X[valid_mask]
y = y[valid_mask]

print(f"✓ Validních vzorků: {len(X):,}")

# Train/test split (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)

print(f"  • Train: {len(X_train):,} vzorků")
print(f"  • Test: {len(X_test):,} vzorků")

# Standardizace
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("✅ Data připravena pro trénování")

In [ ]:
# Trénování modelu

print("🤖 Trénuji Random Forest model...")
print(f"   Parametry: {RF_PARAMS}")

start_time = time.time()

model = MultiOutputRegressor(
    RandomForestRegressor(**RF_PARAMS)
)

model.fit(X_train_scaled, y_train.values)

elapsed = time.time() - start_time

print(f"✅ Trénování dokončeno za {elapsed:.1f}s")

## 📊 7. Evaluace Modelu

In [ ]:
# Evaluace na testovacích datech

print("📊 Evaluace modelu...\n")

y_pred = model.predict(X_test_scaled)

# Metriky pro každý target
results = []

for i, target in enumerate(FUNDAMENTAL_TARGETS):
    y_true_i = y_test.values[:, i]
    y_pred_i = y_pred[:, i]
    
    mae = mean_absolute_error(y_true_i, y_pred_i)
    rmse = np.sqrt(mean_squared_error(y_true_i, y_pred_i))
    r2 = r2_score(y_true_i, y_pred_i)
    
    # Relativní MAE
    mean_val = np.abs(y_true_i).mean()
    mae_pct = (mae / mean_val * 100) if mean_val > 0 else np.nan
    
    results.append({
        'target': target,
        'mae': mae,
        'rmse': rmse,
        'r2': r2,
        'mae_pct': mae_pct
    })

results_df = pd.DataFrame(results)

print("\n📈 VÝSLEDKY PO METRIKÁCH:\n")
display(results_df.style.format({
    'mae': '{:.3f}',
    'rmse': '{:.3f}',
    'r2': '{:.3f}',
    'mae_pct': '{:.1f}%'
}))

print(f"\n📊 CELKOVÝ PRŮMĚR:")
print(f"   • MAE: {results_df['mae'].mean():.3f}")
print(f"   • MAE%: {results_df['mae_pct'].mean():.1f}%")
print(f"   • RMSE: {results_df['rmse'].mean():.3f}")
print(f"   • R²: {results_df['r2'].mean():.3f}")

# Hodnocení
avg_mae_pct = results_df['mae_pct'].mean()
if avg_mae_pct < 15:
    print("\n✨ Výborně! Model dosáhl cílové přesnosti (<15% MAE)")
elif avg_mae_pct < 20:
    print("\n👍 Dobře! Model je použitelný (15-20% MAE)")
else:
    print("\n⚠️ Model má vyšší chybu (>20% MAE)")

In [ ]:
# Vizualizace MAE po metrikách

fig, ax = plt.subplots(figsize=(14, 6))

results_sorted = results_df.sort_values('mae_pct')
colors = ['green' if x < 15 else 'orange' if x < 20 else 'red' for x in results_sorted['mae_pct']]

ax.barh(results_sorted['target'], results_sorted['mae_pct'], color=colors, alpha=0.7)
ax.axvline(x=15, color='green', linestyle='--', linewidth=2, alpha=0.5, label='Cíl: 15%')
ax.set_xlabel('MAE (%)', fontsize=12)
ax.set_title('Relativní MAE pro každou fundamentální metriku', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

## 🔍 8. Feature Importance Analýza

In [ ]:
# Extrakce feature importance

print("🔍 Analýza Feature Importance...\n")

importance_data = []

for i, estimator in enumerate(model.estimators_):
    target = FUNDAMENTAL_TARGETS[i]
    importances = estimator.feature_importances_
    
    for j, feature in enumerate(OHLCV_FEATURES):
        importance_data.append({
            'target': target,
            'feature': feature,
            'importance': importances[j]
        })

importance_df = pd.DataFrame(importance_data)

# Top 5 features pro každý target
print("TOP 5 FEATURES PRO KAŽDÝ TARGET:\n")
for target in FUNDAMENTAL_TARGETS[:5]:  # První 5 pro úsporu místa
    print(f"\n{target}:")
    target_imp = importance_df[importance_df['target'] == target].sort_values('importance', ascending=False)
    display(target_imp.head(5))

In [ ]:
# Vizualizace celkové feature importance

fig, ax = plt.subplots(figsize=(12, 8))

# Průměrná importance přes všechny targety
avg_importance = importance_df.groupby('feature')['importance'].mean().sort_values(ascending=True)

avg_importance.plot(kind='barh', ax=ax, color='steelblue', alpha=0.8)
ax.set_xlabel('Průměrná Importance', fontsize=12)
ax.set_title('Celková Feature Importance (průměr přes všechny fundamenty)', fontsize=14, fontweight='bold')
ax.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

## 💾 9. Uložení Modelu a Výsledků

In [ ]:
# Uložení modelu a scaleru na Google Drive

from joblib import dump

print("💾 Ukládám model a výsledky...")

# Cesty (upravte podle vaší struktury)
MODEL_PATH = '/content/drive/MyDrive/StockPrediction/fundamental_predictor.pkl'
SCALER_PATH = '/content/drive/MyDrive/StockPrediction/feature_scaler.pkl'
METRICS_PATH = '/content/drive/MyDrive/StockPrediction/fundamental_metrics.csv'
IMPORTANCE_PATH = '/content/drive/MyDrive/StockPrediction/feature_importance.csv'

# Uložení
dump(model, MODEL_PATH)
dump(scaler, SCALER_PATH)
results_df.to_csv(METRICS_PATH, index=False)
importance_df.to_csv(IMPORTANCE_PATH, index=False)

print("✅ Uloženo:")
print(f"   • Model: {MODEL_PATH}")
print(f"   • Scaler: {SCALER_PATH}")
print(f"   • Metriky: {METRICS_PATH}")
print(f"   • Feature Importance: {IMPORTANCE_PATH}")

## 🎉 10. Shrnutí

### ✅ Co jsme udělali:

1. ✅ Načetli OHLCV data z Google Drive
2. ✅ Stáhli fundamentální metriky pomocí yfinance (2024-2025)
3. ✅ Spojili OHLCV + fundamenty
4. ✅ Natrénovali Multi-output Random Forest model
5. ✅ Evaluovali model (MAE, RMSE, R²)
6. ✅ Analyzovali feature importance
7. ✅ Uložili model pro další použití

### 📊 Výsledky:

- Průměrná přesnost: **{avg_mae_pct:.1f}% MAE**
- R² score: **{results_df['r2'].mean():.3f}**

### 🔜 Další kroky:

Otevřete **Part 2 Notebook** pro:
- Doplnění historických dat (2015-2024)
- Trénování modelu pro predikci cen
- Finální analýzu a vizualizace